In [1]:
import petl as etl
from collections import OrderedDict, defaultdict
import traceback

In [2]:
# need to configure the source? or assume always a text file for now?
t1 = etl.io.xlsx.fromxlsx(r'c:\data\synth_migration_data_v2.xlsx')
#t1 = etl.select(t1, "{Extr} != 'y'")  # filter out extract eg for now

In [3]:
t1

Site,Sample Name,Notes,Comments,Depth,Depth Unit,Method,Extr,TOC,wt,PeakProperty,Pr/Ph,Custom1,Custom2
Well1,S1,blah,…,2400,m,l,None,8.6,20,Area,0.65,errrm,None
Well1,S2,None,None,2410,m,l,None,8.9,21,None,None,None,232
Well2,S4,but,I'll say it,2215,m,py,None,5.5,13,Conc Area,0.58,None,114
Well2,S5-ex,None,anyway,2215,m,py,y,5.6,12.1,None,None,nah,None
Well3,S6,-,None,4890,f,x,None,2.1,None,None,None,None,None


In [6]:
from collections import namedtuple
from dataclasses import dataclass, field
from typing import Dict

# template code (will use inject compiled parts for registering conversions, 
#                the value mappings inside row_mapper & the headers)
@dataclass(frozen=True)
class ValueDefinitionMapping:
    from_uom: str
    to_uom: str
    from_ratio: str = field(default='NoRatio')
    to_ratio: str = field(default='NoRatio')
        
        
@dataclass(frozen=True)
class ConversionParams:
    a: float
    b: float
    c: float
    d: float


conversions: Dict[ValueDefinitionMapping, ConversionParams] = {}
# <!*** inject conversions here ***>
conversions[ValueDefinitionMapping(from_uom='f', to_uom='m')] = ConversionParams(a=1, b=0, c=0, d=3.2808399)  # used default ratios
conversions[ValueDefinitionMapping(from_uom='m', to_uom='m')] = ConversionParams(a=1, b=0, c=0, d=1)  # used default ratios


def row_mapper(row):
    try:
        #ref_col_src_col_pair_to_dist_ref_col_values
        cache = defaultdict(set)
        
        ###############################################################################################
        # all of this section (inner funcs) could be part of the template or could be define in a  
        # dsl.local_funcs module that is imported by this script - think I prefer the 2nd option
        # ** if doing this consider how to access `row` from `row_mapper` **
        ###############################################################################################
        
        ##################### MAPPING (EXPAND / CONVERT) #####################

        def expand(rc_val, src_col, ref_col):
            cache[(src_col, ref_col)].add(rc_val)
            return row[src_col] if row[ref_col] == rc_val else None

        def expand_default(src_col, ref_col):
            # return src col val (for this row) if ref col val for this row has not been mapped
            return row[src_col] if row[ref_col] not in cache[(src_col, ref_col)] else None

        def convert(vd: ValueDefinitionMapping, value: float):
            p : ConversionParams = conversions.get(vd)
            return ((p.a * value) + p.b) / ((p.c * value) + p.d)

        ############################# COLLAPSE ##############################
        
        # incl all collapse / value mod funcs incase needed or add on demand?
        def append(cols, descs, sep=':', delim=';'):
            """Should be called with cols/descs sorted by pos arg."""
            return delim.join([f"{d if d else c}{sep}{row[c]}" 
                               for d, c in zip(descs, cols) if row[c]])
        
        def pref(cols):
            """Should be called with cols sorted by pref arg."""
            return next((row[col] for col in cols if row[col]), "")
        
        def add(cols):
            return sum(row[col] if row[col] else 0 for col in cols)

        ############################ VALUE MOD #############################
        # define inline within value_funcs module? e.g. within `generate_code()`
        
        ###############################################################################################

        return [
            # <!*** inject mappings here ***> 
            # direct mappings e.g.(mapped col names come later)
            row['Site'],
            row['Sample Name'],

            # do collapse rules (note - generate list here in order based on pos)
            append(cols=['Notes', 'Comments'], descs=[None, None]),

            # conversion  # todo need to map from raw unit col to mapping
            convert(ValueDefinitionMapping(from_uom=row['Depth Unit'], to_uom='m'), value=row['Depth']), 

            # expansion rule 1 
            expand(rc_val='l', src_col='TOC', ref_col='Method'),  # mapping for TOC.leco
            expand(rc_val='py', src_col='TOC', ref_col='Method'),  # mapping for TOC.pyrol
            expand_default(src_col='TOC', ref_col='Method'),  

            # expansion rule 2 
            expand(rc_val='l', src_col='wt', ref_col='Method'),  # mapping for sam wt.leco
            expand(rc_val='py', src_col='wt', ref_col='Method'),  # mapping for sam wt.pyrol
            expand_default(src_col='wt', ref_col='Method'), 

            # exp rule n...
            expand(rc_val='Area', src_col='Pr/Ph', ref_col='PeakProperty'),  # mapping for pr/ph[a]
            expand(rc_val='Conc Area', src_col='Pr/Ph', ref_col='PeakProperty'),  # mapping for sam wt.pyrol
            expand_default(src_col='Pr/Ph', ref_col='PeakProperty'), 
            
            row['Custom1'],
            row['Custom2'],
            #add(['TOC', 'wt'])  # not needed just testing add

            ## TODO -
            #  * Value Mods
            #-trivial  * Remaining collapse rules (pref, add)
            #  * Expand on multiple elements e.g. anal & ind;
            #  * comb collapse / exp / mod  (work out degree neeeded if very difficult) !?!?!
            #  * ? support nested case - caution -> need to support on server as well if we do?
            ]
    except :
        traceback.print_exc()  # log as failure are silent otherwise

# <!*** inject headers ***>
# build list of headers based on expanded col names in IGI prop model (to be dynamically generated)
headers = ['Well Name.Well', 'Name.Sam', 
    'Comments.Sam', 
    'Base MD.Sam', 
    'TOC.leco', 'TOC.pyrol', 'TOC.un', 
    'sam wt.leco', 'sam wt.pyrol','sam wt.un',
    'Pr/Ph[a].Sat-GC', 'Pr/Ph[ca].Sat-GC', 'Pr/Ph[un].Sat-GC', 
    'Custom1', 'Custom2']  # user properties from p:IGI3 may stay unmapped for user to map in p:IGI+
t2 = etl.rowmap(t1, row_mapper, header=headers)
t2

+----------------+----------+----------------------------------+--------------------+----------+-----------+--------+-------------+--------------+-----------+-----------------+------------------+------------------+---------+---------+------+
| Well Name.Well | Name.Sam | Comments.Sam                     | Base MD.Sam        | TOC.leco | TOC.pyrol | TOC.un | sam wt.leco | sam wt.pyrol | sam wt.un | Pr/Ph[a].Sat-GC | Pr/Ph[ca].Sat-GC | Pr/Ph[un].Sat-GC | Custom1 | Custom2 |      |
+================+==========+==================================+====================+==========+===========+========+=============+==============+===========+=================+==================+==================+=========+=========+======+
| 'Well1'        | 'S1'     | 'Notes:blah;Comments:…'          |             2400.0 |      8.6 | None      | None   |          20 | None         | None      |            0.65 | None             | None             | 'errrm' | None    | 28.6 |
+----------------+----------+----------------------------------+--------------------+----------+-----------+--------+-------------+--------------+-----------+-----------------+------------------+------------------+---------+---------+------+
| 'Well1'        | 'S2'     | ''                               |             2410.0 |      8.9 | None      | None   |          21 | None         | None      | None            | None             | None             | None    |     232 | 29.9 |
+----------------+----------+----------------------------------+--------------------+----------+-----------+--------+-------------+--------------+-----------+-----------------+------------------+------------------+---------+---------+------+
| 'Well2'        | 'S4'     | "Notes:but;Comments:I'll say it" |             2215.0 | None     |       5.5 | None   | None        |           13 | None      | None            |             0.58 | None             | None    |     114 | 18.5 |
+----------------+----------+----------------------------------+--------------------+----------+-----------+--------+-------------+--------------+-----------+-----------------+------------------+------------------+---------+---------+------+
| 'Well2'        | 'S5-ex'  | 'Comments:anyway'                |             2215.0 | None     |       5.6 | None   | None        |         12.1 | None      | None            | None             | None             | 'nah'   | None    | 17.7 |
+----------------+----------+----------------------------------+--------------------+----------+-----------+--------+-------------+--------------+-----------+-----------------+------------------+------------------+---------+---------+------+
| 'Well3'        | 'S6'     | 'Notes:-'                        | 1490.4719977344826 | None     | None      |    2.1 | None        | None         | None      | None            | None             | None             | None    | None    |  2.1 |
+----------------+----------+----------------------------------+--------------------+----------+-----------+--------+-------------+--------------+-----------+-----------------+------------------+------------------+---------+---------+------+